!cp '/workspace/repos/agentic-ai/PPM - MCG MADISON RIDGE DST.pdf' /workspace/data
!cp '/workspace/repos/agentic-ai/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx' /workspace/data
!pip3 install llama-index llama-parse llama-agents llama-index-llms-huggingface llama-index-embeddings-huggingface llama-index-llms-huggingface-api llama-index-llms-ollama llama-index-embeddings-ollama
!pip3 install openpyxl sentencepiece protobuf evaluate rouge_score absl-py tensorboardX bitsandbytes peft accelerate python-dotenv dspy-ai
!curl -L https://ollama.com/download/ollama-linux-amd64 -o /usr/bin/ollama
!chmod +x /usr/bin/ollama
!useradd -r -s /bin/false -m -d /usr/share/ollama ollama

!pip3 install InstructorEmbedding

Install tmux, open new screen and run `ollama start`, and then `ollama pull qwen2:1.5b`

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
import dspy
import numpy as np
import pandas as pd
from train_utils import get_csv_string, randomize_row_values, operators_dict, range_description_json, split_df_by_empty_columns, split_df_by_empty_rows, print_trainable_parameters



from llama_index.llms.ollama import Ollama

# model_name = "mistral:latest"
model_name = "llama3.1"
# model_name = "qwen2:1.5b"
# model_name = "gemma:1.5b"
llm = Ollama(model=model_name, url="http://127.0.0.1:11434")
llm.metadata

In [ ]:
!cd /workspace/data && curl -X GET "https://www.ecfr.gov/api/versioner/v1/full/2024-07-23/title-17.xml?chapter=II&part=200&subpart=A&section=200.1" -H "accept: application/xml"

In [ ]:
!cd /workspace/data && curl -X GET "https://www.ecfr.gov/api/versioner/v1/full/2024-07-23/title-17.xml?chapter=II&part=200" -H "accept: application/xml"

In [ ]:
import xml.etree.ElementTree as ET

# Path to your XML file
# xml_file_path = '/workspace/data/section2001.xml'
xml_file_path = '/workspace/data/part200.xml'

# Parse the XML file
tree = ET.parse(xml_file_path)

# Get the root element of the XML document
root = tree.getroot()

In [ ]:
print(out["['1PART 200—ORGANIZATION; CONDUCT AND ETHICS; AND\\nINFORMATION AND REQUESTS\\n', '2Subpart A—Organization and Program Management', '3§ 200.1 General statement and statutory authority.']"])

In [ ]:
def get_list_of_sections(s):
    return s[2:-2].split("', '")

In [ ]:
(['1PART 200—ORGANIZATION; CONDUCT AND ETHICS; AND\\nINFORMATION AND REQUESTS\\n',
  '2Subpart A—Organization and Program Management',
  '3§ 200.1 General statement and statutory authority.'],
 ['1PART 200—ORGANIZATION; CONDUCT AND ETHICS; AND\\nINFORMATION AND REQUESTS\\n',
  '2Subpart A—Organization and Program Management',
  '3§ 200.2 Statutory functions.'])

In [ ]:
test = "['1PART 200—ORGANIZATION; CONDUCT AND ETHICS; AND\\nINFORMATION AND REQUESTS\\n', '2Subpart A—Organization and Program Management', '3§ 200.1 General statement and statutory authority.']"
test2 = "['1PART 200—ORGANIZATION; CONDUCT AND ETHICS; AND\\nINFORMATION AND REQUESTS\\n', '2Subpart A—Organization and Program Management', '3§ 200.2 Statutory functions.']"
get_list_of_sections(test), get_list_of_sections(test2)

In [ ]:
print("METADATA")
for i,t in enumerate(get_list_of_sections(test)):
    if i==0:
        print("[\n",t)
    else:
        print(t)
print("]")
print()
print("TEXT\n", out[test])

In [ ]:
list(out.keys())

In [ ]:
def print_tree(element, depth=0, prev_depth=-1, lists=None, final_dict=None):
    if lists is None:
        lists = [[]]
    if final_dict is None:
        final_dict = {}

    if element.tag == "HEAD":
        # print(depth, prev_depth)
        # Check if depth is increasing
        if depth > prev_depth:
            lists[-1].append(f"{depth}{element.text}")
        else:
            lists.append(lists[-1][:depth-1])
            lists[-1].append(f"{depth}{element.text}")
        
    if element.tag == "P":
        try:
            final_dict[str(lists[-1])] += "\n" + ''.join(element.itertext())
        except:
            final_dict[str(lists[-1])] = ''.join(element.itertext())

    # Recursively call print_tree on each child, increasing the depth
    for child in element:
        print_tree(child, depth + 1, len(lists[-1]), lists, final_dict)

    return final_dict

# Example call to print_tree
out = print_tree(root)

In [ ]:
def print_tree2(element, depth=0):
    # Print current element tag with indentation based on depth
    # print('  ' * depth + f"{depth}{element.tag}")
    if element.tag=="P": 
        ptext = ''.join(element.itertext())
        print(ptext)
    # print('  ' * depth + f"{depth}{element.tag}" +f" {element.text}")
    # Recursively call print_tree on each child, increasing the depth
    for child in element:
        print_tree2(child, depth + 1)

In [ ]:
print_tree2(root)

In [ ]:
# Find all <P> elements
paragraphs = root.findall('.//HEAD')

# Extract and print the text from each <P> element
for p in paragraphs:
    # If you want to include text from child elements (like <b>), use 'itertext()'
    text = ''.join(p.itertext())
    print(text)

In [ ]:
# model_name = "qwen2:1.5b", 
# model_name = "mistral:latest", 32000
model_name, num_ctx = "llama3.1", 128000
llm = dspy.OllamaLocal(model=model_name, max_tokens=4000, num_ctx=num_ctx, model_type='chat')

In [ ]:
filepath = "/workspace/data/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx"
disposition_inputs = [
  "Selling Costs",
  "Disposition Fee",
  "Net Operating Income",
  "Loan Assumption/Payoff",
  "Return of Forecasted Reserves",
  "CF Y 11",
  "Return of Maximum Offering Amount",
  "Projected Terminal Cap Rate",
  "Cash Flows"
]
dfs = pd.read_excel(filepath, sheet_name="5 - Disposition Analysis", header=None)
# Splitting the DataFrame by empty columns
sub_dfs_by_columns = split_df_by_empty_columns(dfs)

# Splitting each sub-DataFrame by empty rows
final_split_dfs = []
for sub_df in sub_dfs_by_columns:
    split_sub_dfs = split_df_by_empty_rows(sub_df)
    final_split_dfs.extend([get_csv_string(x) for x in split_sub_dfs if not x.empty])

dfs.dropna(axis=0, how='all', inplace=True)
dfs.dropna(axis=1, how='all', inplace=True)
fee_columns = ['Disposition Fee', 'Selling Costs']
cashflow_columns = [1,2,3,4,5,6,7,8,9]
ground_truth = dfs[dfs[1].isin(disposition_inputs+cashflow_columns)].iloc[:, :2] # Get only the necessary columns
ground_truth.drop(labels=[16, 17], axis=0, inplace=True) # drop the duplicate Selling and Disposition Costs



In [ ]:
import os
import torch
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.instructor import InstructorEmbedding

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.readers.file import PandasExcelReader
from llama_index.core import set_global_tokenizer, Document
from llama_index.core.embeddings import resolve_embed_model
from transformers import AutoTokenizer
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)


from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

import nest_asyncio
nest_asyncio.apply()

access_token = os.getenv('HF_TOKEN')
llama_api_key = os.getenv('LLAMA_API_KEY')


# filepath = "/workspace/data/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx"
# documents = PandasExcelReader(sheet_name="5 - Disposition Analysis").load_data(filepath)
documents = [Document(text=t) for t in final_split_dfs]

Settings.llm = llm

Settings.chunk_size = 100
Settings.chunk_overlap = 25
# embed_model_name = "BAAI/bge-small-en-v1.5"
embed_model_name = "hkunlp/instructor-base"
# embed_model = HuggingFaceEmbedding(model_name=embed_model_name)
embed_model = InstructorEmbedding(model_name=embed_model_name)
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
# vector_index.storage_context.persist(persist_dir="/workspace/data/storage/alpha")
query_engine = vector_index.as_retriever(similarity_top_k=2)
Settings.embed_model = embed_model


In [ ]:
from dspy.retrieve.llama_index_rm import LlamaIndexRM
retriever = LlamaIndexRM(query_engine)
dspy.settings.configure(lm=llm, rm=retriever)


spreadvars = ['Disposition Fee', 'Net Operating Income', 'Projected Terminal Cap Rate', 'Return of Forecasted Reserves', 'Return of Maximum Offering Amount']
query_str = f"For each of the variables return their value from the spreadsheet in this format 'variable: ' 'value'. If there are any duplicate variables then choose the first instance. Here are the variables: {spreadvars}."
response = query_engine.query(query_str)
print(str(response))

In [ ]:
gt_collect = {}
for row,col in ground_truth.iterrows():
    if isinstance(col.values[0], int):
        name = f"Cashflows {col.values[0]}"
    else:
        name = col.values[0]
    value = col.values[1]
    gt_collect[name] = str(value)

In [ ]:
# import random
# dfs_str = get_csv_string(dfs)
num_rounds = 2
train_data = []
for _ in range(num_rounds):
    # TODO: gradually increase n_samples, random fill in of values in range
    # dfs_aug = randomize_row_values(dfs, ground_truth=ground_truth, n_samples=15)
    # dfs_str = get_csv_string(dfs_aug)
    # dfs_str = get_csv_string(dfs)
    
    for value_to_extract in gt_collect:

        question = f"Extract the value for the variable name '{value_to_extract}'."
        answer = f"{value_to_extract}: {gt_collect[value_to_extract]}"
        train_data.append(dspy.Example(question=question, answer=answer).with_inputs('question'))
        # train_data.append(dspy.Example(long_text=question, answer=answer).with_inputs('question'))
    
# random.shuffle(train_data)


In [ ]:
%load_ext autoreload
%autoreload 2

from train_utils import operators_dict, range_description_json
from models_testing import SpreadSheetAnalyzer
spreadsheeet_ananlyst = SpreadSheetAnalyzer(range_description_json, operators_dict, num_passages=3)

In [ ]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, BootstrapFinetune, BootstrapFewShot, MIPRO, MIPROv2

perc_train = 0.7
num_train = int(len(train_data) * perc_train)
def validate_answer(pred, example, trace=None):
    return example.answer.lower() == pred.answer.lower()
metric = dspy.evaluate.metrics.answer_exact_match
# metric = dspy.evaluate.metrics.answer_passage_match
# metric = validate_answer
NUM_THREADS=1
TRAIN_NUM=17


#Compile program on BootstrapFinetune
finetune_optimizer = MIPROv2(prompt_model=llm, task_model=llm, metric=metric, num_candidates=10, init_temperature=1.2, minibatch_size=1)
kwargs = dict(num_threads=NUM_THREADS, display_progress=True, display_table=0)
finetune_program = finetune_optimizer.compile(spreadsheeet_ananlyst, trainset=train_data[:TRAIN_NUM], 
                                                 num_batches=20, max_bootstrapped_demos=3, max_labeled_demos=5, 
                                                 eval_kwargs=kwargs, requires_permission_to_run=False)

# finetune_optimizer = MIPRO(prompt_model=llm, task_model=llm, metric=metric, num_candidates=10, init_temperature=1.2)
# kwargs = dict(num_threads=NUM_THREADS, display_progress=True, display_table=0)
# compiled_prompt_opt = finetune_optimizer.compile(spreadsheeet_ananlyst, trainset=train_data[:TRAIN_NUM], num_trials=100, 
#                                                  max_bootstrapped_demos=3, max_labeled_demos=5, eval_kwargs=kwargs,
#                                                  requires_permission_to_run=False)

# config = dict(epochs=3, bf16=True, bsize=1, accumsteps=3, lr=7e-5) #path_prefix=None
# finetune_optimizer = BootstrapFinetune(metric=metric)
# finetune_program = finetune_optimizer.compile(spreadsheeet_ananlyst, trainset=train_data[:20], **config)

# finetune_optimizer = BootstrapFewShot(metric=metric, max_bootstrapped_demos=8, max_labeled_demos=8)
# finetune_program = finetune_optimizer.compile(spreadsheeet_ananlyst, trainset=train_data)

# #Load program and activate model's parameters in program before evaluation
# ckpt_path = "saved_checkpoint_path_from_finetuning"
# LM = dspy.HFModel(checkpoint=ckpt_path, model=model_name)

# for p in finetune_program.predictors():
#     p.lm = LM
#     p.activated = False

In [ ]:
# dfs_str = get_csv_string(dfs)
collection = []
for value_to_extract in gt_collect:
    # if value_to_extract=="Selling Costs":
        # continue
    question = f"Extract the value for the variable name '{value_to_extract}'?"
    print(question)
    pred = finetune_program(question, verbose=True)
    print(pred.answer)
    collection.append((pred, f"{value_to_extract}: {gt_collect[value_to_extract]}"))

In [ ]:
%load_ext autoreload
%autoreload 2

from train_utils import operators_dict, range_description_json
from models_testing import SpreadSheetAnalyzer
spreadsheeet_ananlyst = SpreadSheetAnalyzer(range_description_json, operators_dict, num_passages=3)

In [ ]:
# dfs_str = get_csv_string(dfs)
collection = []
for value_to_extract in gt_collect:
    # if value_to_extract=="Selling Costs":
        # continue
    question = f"Extract the value for the variable name '{value_to_extract}'."
    print(question)
    pred = spreadsheeet_ananlyst(question, verbose=True)
    print(pred.answer)
    collection.append((pred, f"{value_to_extract}: {gt_collect[value_to_extract]}"))

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="value_retriever",
        description="provides useful information about a query.",
    ),
)

# query_engine_tools=[]

In [ ]:
from typing import List, Literal
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.tools import FunctionTool

def adding_values(values: List[float]):
    return sum(values)

class AddingArgs(BaseModel):
    values: List = Field(
        description="A list of values to add together."
    )

adding_tool = FunctionTool.from_defaults(
    fn=adding_values,
    name="sum_values",
    description="Add values together",
    fn_schema=AddingArgs,
)

# query_engine_tools = [adding_tool]


In [ ]:
from llama_agents import (
    AgentService,
    ToolService,
    LocalLauncher,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import ReActAgentWorker, ReActAgent

# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=Settings.llm),
    vector_store=vector_index.storage_context.vector_store,
    # port=8001
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=[query_engine_tools],
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in [query_engine_tools]
]


# define Agent and agent service
worker1 = FunctionCallingAgent.from_tools(
worker1 = ReActAgentWorker.from_tools(
# worker1 = ReActAgent.from_tools(
    # [query_engine_tools],
    meta_tools,
    # tool_service,
    llm=hf_llm,
    # max_iterations=15
)
agent1 = worker1.as_agent()
# agent_server_1 = AgentService(
#     agent=agent1,
#     message_queue=message_queue,
#     description="Used to retrieve values from a spreadsheet that has been converted to a string.",
#     service_name="spreadsheet_reader_agent",
# )

In [ ]:
worker2 = ReActAgent.from_llm(llm=hf_llm)

In [ ]:
response = worker2.query("What is the value for Selling Costs? It is a value between 0 and 1.")

In [ ]:
print(response.response)

In [ ]:
launcher = LocalLauncher(
    [agent_server_1, tool_service],
    control_plane,
    message_queue,
)
query_str = "What is the Disposition Fee?"
result = launcher.launch_single(query_str)

In [ ]:
import torch
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.readers.file import PandasExcelReader
from llama_index.core import set_global_tokenizer
from llama_index.core.embeddings import resolve_embed_model
from transformers import AutoTokenizer
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

import nest_asyncio
nest_asyncio.apply()

# set_global_tokenizer(
#     AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha").encode
# )



filepath = "/workspace/data/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx"
documents = PandasExcelReader(sheet_name="5 - Disposition Analysis").load_data(filepath)
# documents[0].text = documents[0].text.split("\n")
# import os
# os.environ['HF_TOKEN']=access_token
# model_name = "jmars/trithemius-mistral-0.3-7b"
# model_name = "Qwen/Qwen2-1.5B-Instruct"


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "Qwen/Qwen2-1.5B-Instruct"
tokenizer_name = model_name
model = AutoModelForCausalLM.from_pretrained(model_name)
hf_llm = HuggingFaceLLM(model=model, tokenizer_name=tokenizer_name, is_chat_model=True)
Settings.llm = hf_llm


In [ ]:
from typing import List, Literal
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.tools import FunctionTool
from llama_index.core.base.llms.types import (
    ChatMessage,
    MessageRole,
)


def adding_values(values: List[float]):
    return sum(values)


class AddingArgs(BaseModel):
    values: List = Field(
        description="A list of values to add together."
    )

adding_tool = FunctionTool.from_defaults(
    fn=adding_values,
    name="sum_values",
    description="Add a list of values together",
    fn_schema=AddingArgs,
)

data=documents[0].text
usr_msg = ChatMessage(
    role=MessageRole.USER,
    # content=f"What is the sum of Disposition Fee percentage and Sales Cost percentage from this spreadsheet?\n\n##SPREADSHEET\n{data}",
    content=f"Extract the percentage values for 'Disposition Fee' and 'Sales Cost' from this spreadsheet?\n\n##SPREADSHEET\n{data}",
)

response = hf_llm.chat(
    messages=[usr_msg],
    tools=[
        adding_tool
    ],
    tool_choice="add_values",
)

print(response.message)

In [ ]:
dir(response)

In [ ]:
print(response.message)

In [ ]:
import torch
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.readers.file import PandasExcelReader
from llama_index.core import set_global_tokenizer
from llama_index.core.embeddings import resolve_embed_model
from transformers import AutoTokenizer
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

import nest_asyncio
nest_asyncio.apply()

import os
os.environ['HF_TOKEN']=access_token

# model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model_name = "Qwen/Qwen2-1.5B-Instruct"
tokenizer_name = model_name
embed_model_name = "BAAI/bge-small-en-v1.5"

hf_llm = HuggingFaceLLM(model_name=model_name, tokenizer_name=tokenizer_name, is_chat_model=True, device_map='auto', max_new_tokens=2000, context_window=8000)
set_global_tokenizer(
    AutoTokenizer.from_pretrained(tokenizer_name).encode  # pass in the HuggingFace model org + repo
)
# hf_llm = HuggingFaceLLM(model_name=model_name, tokenizer_name=tokenizer_name)
# hf_llm = HuggingFaceInferenceAPI(model_name=model_name, tokenizer_name=tokenizer_name, is_chat_model=True, is_function_calling_model=True)
embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

filepath = "/workspace/data/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx"
documents = PandasExcelReader(sheet_name="5 - Disposition Analysis").load_data(filepath)

vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
# vector_index.storage_context.persist(persist_dir="/workspace/data/storage/alpha")
query_engine = vector_index.as_query_engine(llm=hf_llm, top_k=3)

Settings.llm = hf_llm
Settings.embed_model = embed_model

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from typing import List, Literal
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.tools import FunctionTool
from llama_index.core.base.llms.types import (
    ChatMessage,
    MessageRole,
)


query_engine_tools = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="spreadsheet_value_retriever",
        description="contains the information of a spreadsheet, and is useful for retrieving specific values from a spreadsheet",
    ),
)

def adding_values(values: List[float]):
    return sum(values)


class AddingArgs(BaseModel):
    values: List = Field(
        description="A list of values to add together."
    )

adding_tool = FunctionTool.from_defaults(
    fn=adding_values,
    name="sum_values",
    description="Add a list of values together",
    fn_schema=AddingArgs,
)

In [ ]:
from llama_agents import (
    AgentService,
    ToolService,
    LocalLauncher,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import ReActAgentWorker, ReActAgent



# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=hf_llm),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=[query_engine_tools],#, adding_tool],
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in [query_engine_tools]#, adding_tool]
]


# define Agent and agent service
# worker1 = FunctionCallingAgentWorker.from_tools(
worker1 = ReActAgentWorker.from_tools(
    meta_tools,
    llm=hf_llm,
)
agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Used to answer questions over Uber and Lyft 10K documents",
    service_name="uber_lyft_10k_analyst_agent",
)

In [ ]:
launcher = LocalLauncher(
    [agent_server_1, tool_service],
    control_plane,
    message_queue,
)
query_str = "What is the Disposition Fee?"
result = launcher.launch_single(query_str)

In [ ]:
from llama_index.core.agent import ReActAgentWorker, ReActAgent
agent = ReActAgent.from_tools(
    [query_engine_tools, adding_tool],
    llm=hf_llm,
    verbose=True,
    )

In [ ]:
message = ['Selling Costs',
  'Disposition Fee',
  'Net Operating Income',
  'Loan Assumption/Payoff',
  'Return of Forecasted Reserves',
  'CF Y 11',
  'Return of Maximum Offering Amount',
  'Projected Terminal Cap Rate',
  'Cash Flows']
content='Retrieve the following values from the spreadsheet: Selling Costs, Disposition Fee, Net Operating Income, Loan Assumption/Payoff, Return of Forecasted Reserves, CF Y 11, Return of Maximum Offering Amount, Projected Terminal Cap Rate, Cash Flows (categories 1 through 9)\nThen add Disposition Fee and Selling Cost together.'

usr_msg = ChatMessage(
    role=MessageRole.ASSISTANT,
    content=content
)
response = agent1.chat(content)

In [ ]:
response

In [ ]:
content='Retrieve the following values from the spreadsheet: Selling Costs, Disposition Fee, Net Operating Income, Loan Assumption/Payoff, Return of Forecasted Reserves, CF Y 11, Return of Maximum Offering Amount, Projected Terminal Cap Rate, Cash Flows (categories 1 through 9)\nThen add Disposition Fee and Selling Cost together.'

messages = [
        {"role": "user", "content": content},
    ]

In [ ]:
#  Forward NOI Growth 	2.00%
#  Selling Costs 	1.00%
#  Disposition Fee 	2.50%
	
# 	Assumes. 0-yr Hold
# 	Scenario A
# Net Operating Income	 4,644,391 
# Projected Terminal Cap Rate	5.25%
# Projected Sales Price (95%)	 88,464,592 
# Loan Assumption/Payoff	 -   
# Selling Costs	 (884,646)
# Disposition Fee	 (2,211,615)
# Return of Forecasted Reserves	 -   
# Sale Proceeds	 85,368,331 
# Proceeds from Distributions	 36,688,942 
# Return of Maximum Offering Amount	 (77,670,567)
# DST Total Gain / (Loss)	 44,386,707 

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import logging

# create a tool
def get_the_secret_fact() -> str:
    """Returns the secret fact."""
    return "The secret fact is: A baby llama is called a 'Cria'."

tool = FunctionTool.from_defaults(fn=get_the_secret_fact)

# Define an agent
model_name = "Qwen/Qwen2-1.5B-Instruct"
llm = HuggingFaceLLM(model_name=model_name)
worker = FunctionCallingAgentWorker.from_tools([tool], llm=llm)
agent = worker.as_agent()

# Create an agent service
agent_service = AgentService(
    agent=agent,
    message_queue=message_queue,
    description="General purpose assistant",
    service_name="assistant",
)